# Данные клиентов страховой компании

Нужно защитить данные клиентов страховой компании. Требуется:
1. Разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию.
2. Обосновать корректность работы метода.
3. Обеспечить защиту данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось.

*Подбирать наилучшую модель не требуется.

## 1. Загрузка данных

###  1.1. Импорт модулей

In [ ]:
import requests
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from urllib.parse import urlencode
from sklearn.metrics import r2_score
from mpl_toolkits.mplot3d import Axes3D
from sklearn.linear_model import LinearRegression

In [ ]:
LINKS = ["https://yadi.sk/d/opCKECxgXuV0SQ"]
PATHS = ['/datasets/insurance.csv']
RANDOM = 12345

###  1.2. Функции

In [ ]:
def load_data(paths, links=None):
    """
    Функция принимает пути до локальных файлов с данными и (опционально) ссылки
    для их скачивания.
    
    paths - локальные пути до файлов с данными
    links - ссылки для скачивания (Яндекс.Диск)
    """
    
    data_list = []
    base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
    
    if links != None and not links:
        raise Exception("Error: wrong 'links' value (nmust be not empty list)")
    
    if links:
        if len(paths) != len(links):
            raise Exception("Error: length of 'links' must be equal to length of 'paths'")

        if type(paths).__name__ != 'list' or type(links).__name__ != 'list':
            raise Exception("Error: variables 'links' and 'paths' must be 'list' type")

    for i in range(len(paths)):
        
        try:
            path = paths[i]
            data_list.append(pd.read_csv(path))
            
        except FileNotFoundError:
            # download from yandex disk
            public_key = links[i]
            print("Run load", public_key)
            
            # get download link
            final_url = base_url + urlencode(dict(public_key=public_key))
            response = requests.get(final_url)
            download_url = response.json()['href']
            data = pd.read_csv(download_url)
            data_list.append(pd.DataFrame(data))
            print('Done.')
            
    return data_list

In [ ]:
def learn(X, y, show_matrix=True):
    if show_matrix:
        print(f'The matrix {X.shape}:')
        print(X[:10])
    m = LinearRegression()
    m.fit(X, y)
    p = m.predict(X)
    print('\nR2:', r2_score(y, p))

In [ ]:
def check_inverse(matrix):
    try:
        _ = np.linalg.inv(matrix)
        print('OK: The matrix is invertible\n')
    except np.linalg.LinAlgError as e:
        if 'Singular matrix' in str(e):
            print('WARN: The matrix is not invertible\n')
        else:
            raise

In [ ]:
def get_weight(X, y):
    return np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)

###  1.3. Данные

In [ ]:
insurance_data = load_data(PATHS, links=LINKS)[0]

In [ ]:
insurance_data.info()

In [ ]:
# insurance_data = insurance_data.set_axis(['sex', 'age', 'salary', 'family', 'payments'], axis=1)
insurance_data.columns = ['sex', 'age', 'salary', 'family', 'payments']
insurance_data

In [ ]:
# признаки и целевая переменная
features = insurance_data.drop('payments', axis=1)
target = insurance_data['payments']

# матрица признаков и целевой признак
feature_matrix = features.values
target_matrix = target.values

feature_matrix.shape

## 2. Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Ответьте на вопрос и обоснуйте решение. Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

1. Изменится. Приведите примеры матриц.

2. Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.

**Ответ:** Не изменится.

**Обоснование:**

Распишем формулу обучения (получения весов) с учетом умножения на матрицу ***P***.

$$
((X P)^T (X P))^{-1} (X P)^T y = (X^T P^T X P)^{-1} X^T P^T y = (X^T)^{-1} (P^T)^{-1} (X)^{-1} (P)^{-1} X^T P^T y = (X^T X)^{-1} (P^T P)^{-1} X^T P^T y = ((P^T P)^{-1} P^T) ((X^T X)^{-1} X^T) y = ((P^T P)^{-1} P^T) w = (1)
$$

Полученное выражение (1) подставим в формулу предсказания (также с учетом умножения на матрицу **P**):

$$
a = X [ P ((P^T P)^{-1} P^T) ] w = X [ P (P^T)^{-1}P^{-1}P^T ] w = X [((P^T)^{-1} P^T) (P^{-1} P)] w = X [(P^{-1} P)^T E] w = X [ E^T E ] w = X E E w = X w
$$

Соответственно, для возможности вычислить веса в задаче линейной регрессии (1), необходимо, чтобы матрица ***P*** была обратимой.

В задаче матрицы признаков имеет размер 5000 x 4 => для удовлетворения условию (умножение + обратимость искомой матрицы) нужна квадратная матрица 4 x 4.

Рассмотрим 3 случая матриц, потенциально пригодных для использования в преобразовании:

1. Матрица, у которой все элементы равны 1
2. Единичная матрица (единицы на главной диагонали)
3. Случайная обратимая матрица

**1. Матрица из единиц *M*:**

$$
M = \begin{pmatrix} 1& 1& 1& 1&\\ 1& 1& 1& 1&\\ 1& 1& 1& 1&\\ 1& 1& 1& 1&\\ \end{pmatrix}
$$

$$
M E = M
$$

Определитель такой матрицы равен 0 => матрица вырожденная и необратима, так как у нее нет определителя:

$$
det A^{-1} = (det A)^{-1}
$$

Соответственно, преобразование признаков путем умножения на такую матрицу если не приведет к ошибке кода, то окажет непредсказуемое влияние на точность модели.

**2. Единичная матрица *E*:**

$$
M = \begin{pmatrix} 1& 0& 0& 0&\\ 0& 1& 0& 0&\\ 0& 0& 1& 0&\\ 0& 0& 0& 1&\\ \end{pmatrix}
$$

По свойствам обратной матрицы к единичной:

$$
E^{-1} = E
$$

Преобразование признаков путем умножения на такую матрицу не окажет влияния на точность модели, но и не приведет к изменению признаков:

$$
A E = A
$$

**3. Случайная обратимая матрица**

В случае использования преобразования признаков такой матрицей, будут выполняться все алгебраические свойства обратимости и транспонирования матриц, что позволит зашифровать значения признаков, тем самым не повлияв на точность модели.

Воспроизведем в коде описанное выше.

In [ ]:
# 1. Исходная матрицы признаков

learn(feature_matrix, target_matrix)

In [ ]:
# 2. Матрица из единиц

ones_matrix = np.ones((4,4))
check_inverse(ones_matrix)

feature_matrix_ones = feature_matrix.dot(ones_matrix)
learn(feature_matrix_ones, target_matrix)

**Вывод**

Признаки преобразованы. Ошибки в коде нет, однако качество модели заметно изменилось. Такая матрица не подходит.

In [ ]:
# 3. Единичная матрица

eye_matrix = np.eye(4, 4)
check_inverse(eye_matrix)

feature_matrix_eye = feature_matrix.dot(eye_matrix)
learn(feature_matrix_eye, target_matrix)

**Вывод**

Признаки не изменились, качество модели не изменилось. Матрица не подходит.

In [ ]:
# 3. Случайная обратимая матрица

random_matrix = np.random.random((4,4))
check_inverse(random_matrix)

feature_matrix_rand = feature_matrix.dot(random_matrix)
learn(feature_matrix_rand, target_matrix)

**Вывод**

Признаки изменились, качество модели не изменилось. Матрица подходит для преобразования.

In [ ]:
# Веса, полученные в ходе экспериментов

_, ax = plt.subplots(1, 3, figsize=(9,3))

for axi, matrix, c, title in zip(ax, 
                                 [feature_matrix, feature_matrix_eye, feature_matrix_rand], 
                                 ['r', 'b', 'g'], 
                                 ['Original', 'E', 'Random']):
    axi.scatter(
        get_weight(matrix, target_matrix),
        y=get_weight(feature_matrix, target_matrix),
        color=c
    )
    axi.set_xlabel('target')
    axi.set_ylabel('target')
    axi.set_title(title)
    
plt.tight_layout()
plt.show()

In [ ]:
X3 = TSNE(n_components=3).fit_transform(feature_matrix)
X3_eye = TSNE(n_components=3).fit_transform(feature_matrix_eye)
X3_rand = TSNE(n_components=3).fit_transform(feature_matrix_rand)

print(X3.shape)
print(X3_eye.shape)
print(X3_rand.shape)

In [ ]:
fig = plt.figure(figsize=(12,10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X3[:,0], X3[:,1], X3[:,2], color='#3498DB', alpha=0.3)
ax.scatter(X3_eye[:,0], X3_eye[:,1], X3_eye[:,2], color='#F4D04F', alpha=0.3)
ax.scatter(X3_rand[:,0], X3_rand[:,1], X3_rand[:,2], color='#EC7063', alpha=0.3)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.legend(['feature_matrix', 'feature_matrix_eye', 'feature_matrix_rand'])
plt.show()

На графике видно, что после прообразования расположение признаков в пространстве совершенно изменилось.

## 3. Алгоритм преобразования

**Алгоритм**

В качестве алгоритма преобразования данных выбирается случайная матрица, которая является обратимой - *feature_matrix_rand*.

**Обоснование**

Алгоритм выбран, так как:
1. Данные меняются и их невозможно интепретировать и связать с реальными данными (цель задачи).
2. Матрица обратимая, следовательно, данные возжно расшифровать с помощью обратной матрицы преобразования.
3. Преобразование не влияет на качество модели, соответственно, не привносится искажение результатов работы модели.

## 4. Проверка алгоритма

In [ ]:
print('До преобразования')
learn(feature_matrix, target_matrix, show_matrix=False)

In [ ]:
print('После преобразования')
learn(feature_matrix_rand, target_matrix, show_matrix=False)

## Вывод

Метрики качества моделей до и после преобразования признаков отличаются после 12 знака после запятой, что является несущественным отличием. Соответственно, выбранный алгоритм преобразования данных пригоден для шифрования данных без последствий для модели машинного обучения.

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные загружены
- [x]  Выполнен шаг 2: получен ответ на вопрос об умножении матриц
    - [x]  Указан правильный вариант ответа
    - [x]  Вариант обоснован
- [x]  Выполнен шаг 3: предложен алгоритм преобразования
    - [x]  Алгоритм описан
    - [x]  Алгоритм обоснован
- [x]  Выполнен шаг 4: алгоритм проверен
    - [x]  Алгоритм реализован
    - [x]  Проведено сравнение качества моделей до и после преобразования